# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,singkil,-7.9855,111.4161,20.94,93,100,1.86,ID,1722885784
1,1,tomatlan,19.9333,-105.2500,34.21,49,24,3.01,MX,1722885786
2,2,usinsk,65.9939,57.5281,10.25,72,99,5.75,RU,1722885787
3,3,edinburgh of the seven seas,-37.0676,-12.3116,13.74,91,92,11.02,SH,1722885788
4,4,longyearbyen,78.2186,15.6401,9.91,81,20,6.17,SJ,1722885789


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
weather_df = city_data_df
weather_df = weather_df.loc[(weather_df["Max Temp"] < 27) & (weather_df["Max Temp"] > 21)]
weather_df = weather_df.loc[(weather_df["Wind Speed"] < 4.5)]
weather_df = weather_df.loc[(weather_df["Cloudiness"] == 0)]

# Drop any rows with null values
weather_df = weather_df.dropna()

# Display sample data
weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,57,tanaina,61.6269,-149.4281,21.58,60,0,0.45,US,1722885854
68,68,el granada,37.5027,-122.4694,23.60,73,0,2.57,US,1722885867
80,80,wenling,28.3669,121.3600,26.27,88,0,1.20,CN,1722885880
84,84,swift current,50.2834,-107.8014,24.45,38,0,3.09,CA,1722885885
94,94,tongliao,43.6125,122.2653,21.79,64,0,2.98,CN,1722885896
97,97,extrema,-22.8547,-46.3183,26.91,30,0,1.97,BR,1722885899
119,119,hasaki,35.7333,140.8333,25.22,90,0,2.39,JP,1722885912
157,157,katsuura,35.1333,140.3000,24.54,92,0,2.85,JP,1722885969
162,162,buka,40.8108,69.1986,24.46,25,0,1.49,UZ,1722885974
169,169,meadow lakes,61.6247,-149.6011,21.81,63,0,0.00,US,1722885982


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
57,tanaina,US,61.6269,-149.4281,60,
68,el granada,US,37.5027,-122.4694,73,
80,wenling,CN,28.3669,121.3600,88,
84,swift current,CA,50.2834,-107.8014,38,
94,tongliao,CN,43.6125,122.2653,64,
97,extrema,BR,-22.8547,-46.3183,30,
119,hasaki,JP,35.7333,140.8333,90,
157,katsuura,JP,35.1333,140.3000,92,
162,buka,UZ,40.8108,69.1986,25,
169,meadow lakes,US,61.6247,-149.6011,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
    

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tanaina - nearest hotel: Alaska's Select Inn Hotel
el granada - nearest hotel: Beach House
wenling - nearest hotel: Koro
swift current - nearest hotel: York Hotel & Bar
tongliao - nearest hotel: No hotel found
extrema - nearest hotel: Hotel San Marco
hasaki - nearest hotel: Choshi Plaza Hotel
katsuura - nearest hotel: 潮騒の宿
buka - nearest hotel: No hotel found
meadow lakes - nearest hotel: Best Western Lake Lucille Inn
masuguru - nearest hotel: No hotel found
alpatovo - nearest hotel: No hotel found
nahrin - nearest hotel: No hotel found
fuxin - nearest hotel: 凯文国际商务酒店
la roche-sur-yon - nearest hotel: Hôtel Mercure La Roche Sur Yon Centre
huai'an - nearest hotel: 神旺大酒店
tongchuanshi - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
villa del totoral - nearest hotel: No hotel found
malanje - nearest hotel: Hotel PORTVGALIA


,City,Country,Lat,Lng,Humidity,Hotel Name
57,tanaina,US,61.6269,-149.4281,60,Alaska's Select Inn Hotel
68,el granada,US,37.5027,-122.4694,73,Beach House
80,wenling,CN,28.3669,121.3600,88,Koro
84,swift current,CA,50.2834,-107.8014,38,York Hotel & Bar
94,tongliao,CN,43.6125,122.2653,64,No hotel found
97,extrema,BR,-22.8547,-46.3183,30,Hotel San Marco
119,hasaki,JP,35.7333,140.8333,90,Choshi Plaza Hotel
157,katsuura,JP,35.1333,140.3000,92,潮騒の宿
162,buka,UZ,40.8108,69.1986,25,No hotel found
169,meadow lakes,US,61.6247,-149.6011,63,Best Western Lake Lucille Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True,
    tiles = "OSM", 
    frame_width = 900, 
    frame_height = 700, 
    color = "City", 
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)